<div class="alert alert-block alert-info"> <b>NOTE</b> Please select the kernel <code>Python [conda env:gnn-tf]</code> for this notebook. </div>

# 2. Inference - Segmentation

In [1]:
import os
import re
import subprocess

In [1]:
import tensorflow as tf

2024-07-09 16:32:34.524997: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
tf.config.list_physical_devices('GPU')

2024-07-09 16:32:37.646425: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-07-09 16:32:37.647893: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-07-09 16:32:37.695827: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2024-07-09 16:32:37.696525: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:23:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2024-07-09 16:32:37.697077: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:41:00.0 name: NVIDIA RTX A60

[]

In [2]:
software_dir = 'software'
data_dir = 'ctc-data/pred-seg'

models = [
    'Fluo-N2DL-HeLa'
]

pattern = re.compile('\d{3}')
data_ids = [f for f in os.listdir(os.path.join(data_dir, models[0])) if pattern.fullmatch(f)]

Predict segmentations

In [3]:
fov = '25'
min_size = '10'
max_size = '1000000'
edge_dist = '2'
sigmoid_thresh = '0.4'
min_center_size = '5'
pre_sequence_frames = '4'
data_format = 'NCHW'

code_seg = os.path.join(software_dir, 'seg_code/new_version')
seg_model_dir = os.path.join(software_dir, 'parameters/Seg_Models')

In [5]:
for m in models:
    print('------', m, '------')
    for i in data_ids:
        print(i)
        p = subprocess.run(
            ['python', os.path.join(code_seg, 'Inference2D.py'),
             '--gpu', '0',
             '--model_path', os.path.join(seg_model_dir, m),
             '--sequence_path', os.path.join(data_dir, m, i),
             '--output_path', os.path.join(data_dir, m, f'{i}_SEG_RES'),
             '--edge_dist', edge_dist,
             '--min_cell_size', min_size,
             '--max_cell_size', max_size,
             '--fov', fov,
             '--centers_sigmoid_threshold', sigmoid_thresh,
             '--min_center_size', min_center_size,
             '--pre_sequence_frames', pre_sequence_frames,
             '--data_format', data_format,
            #  '--save_intermediate',
            #  '--save_intermediate_path', os.path.join(data_dir, m, f'{i}_SEG_intermediate')
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE)

        print(p.stdout.decode("utf-8").strip())
        print(p.stderr.decode("utf-8").strip())
        print('\n')

------ Fluo-N2DL-HeLa ------
001
2024-07-09 16:23:38: Restored from software/parameters/Seg_Models/Fluo-N2DL-HeLa/model.ckpt
Done!
2024-07-09 16:23:39: Done
2024-07-09 16:23:36.663970: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2024-07-09 16:23:38.114441: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-07-09 16:23:38.115316: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-07-09 16:23:38.158826: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA RTX A6000 computeCapability: 8.6
coreClock: 1.8GHz coreCount: 84 deviceMemorySize: 47.54GiB deviceMemoryBandwidth: 715.34GiB/s
2024-07-09 16:23:38.158857: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.s